In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy
from neuralnilm.data.datathread import DataThread

In [2]:
NILMTK_FILENAME = '/data/mine/vadeec/merged/ukdale.h5'
TARGET_APPLIANCE = 'kettle'
SEQ_LENGTH = 256
SAMPLE_PERIOD = 6
STRIDE = 4
WINDOWS = {
    'train': {
        1: ("2014-01-01", "2014-02-01")
    },
    'unseen_activations_of_seen_appliances': {
        1: ("2014-02-02", "2014-02-08")                
    },
    'unseen_appliances': {
        2: ("2013-06-01", "2013-06-07")
    }
}

LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename=NILMTK_FILENAME,
        sample_period=SAMPLE_PERIOD,
        windows=WINDOWS
    )
}



In [3]:
nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

INFO:neuralnilm.data.loadactivations:Loading NILMTK activations...
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 111 kettle activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 119 microwave activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 23 washing machine activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 kettle activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 30 microwave activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading washing mac

In [4]:
nilmtk_activations.keys()

['train', 'unseen_activations_of_seen_appliances', 'unseen_appliances']

In [5]:
from neuralnilm.data.realaggregatesource import RealAggregateSource

In [6]:
ras = RealAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    filename=NILMTK_FILENAME,
    windows=WINDOWS,
    sample_period=SAMPLE_PERIOD
)

INFO:neuralnilm.data.realaggregatesource:Loading NILMTK mains...
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold train from 2014-01-01 00:00:00+00:00 to 2014-01-31 23:59:54+00:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_2...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_2 for fold unseen_appliances from 2013-06-01 00:00:00+01:00 to 2013-06-06 23:59:54+01:00.
INFO:neuralnilm.data.realaggregatesource:Loading mains for UK-DALE_building_1...
INFO:neuralnilm.data.realaggregatesource:Loaded mains data from building UK-DALE_building_1 for fold unseen_activations_of_seen_appliances from 2014-02-02 00:00:00+00:00 to 2014-02-07 23:59:54+00:00.
INFO:neuralnilm.data.realaggregatesource:Done loading NILMTK mains data.
INFO:neuralnilm.data.realaggregatesource:Found 89 sections without targ

In [7]:
ras.target_inclusion_prob = 0.5
for i in range(50):
    seq = ras.get_sequence()

In [8]:
fig, axes = plt.subplots(2)
axes[0].plot(seq.input)
axes[1].plot(seq.target)
plt.show()

In [9]:
nilmtk_activations['train']['kettle']['UK-DALE_building_1'][0]._metadata

['name', 'building', 'appliance', 'fold']

In [10]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    allow_incomplete_target=False,
    sample_period=SAMPLE_PERIOD
)

In [11]:
FOLD = 'train'
#FOLD = 'unseen_activations_of_seen_appliances'
#FOLD = 'unseen_appliances'
seq = source.get_sequence(enable_all_appliances=True)

In [12]:
#all(seq.all_appliances.sum(axis=1) == seq.input[:, 0])

In [13]:
#fig, axes = plt.subplots(2, sharex=True)
#seq.all_appliances.plot(ax=axes[0])
#axes[1].plot(seq.input)
#fig.suptitle(FOLD)
#plt.show()

In [14]:
sample = source.get_batch(num_seq_per_batch=1024)

In [15]:
pipeline = DataPipeline(
    sources=[source, ras],
    num_seq_per_batch=64,
    input_processing=[DivideBy(sample.before_processing.input.flatten().std())],
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

# Disaggregation

nilmtk_disag_source = NILMTKDisagSource(
    filename=NILMTK_FILENAME,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    buildings=[5],
    window_per_building={},
    stride=STRIDE,
    sample_period=SAMPLE_PERIOD
)

disag_pipeline = deepcopy(pipeline)
disag_pipeline.source = nilmtk_disag_source

disaggregator = Disaggregator(
    pipeline=disag_pipeline,
    output_path=PATH  # 
)

Disagregator ideas:

* make a copy of pipeline but swap source for a NILMTKDisagSource
* NILMTKDisagSource loads all data into memory (?) and iterates over chunks of it (get seq_length from pipeline.source.seq_length)
* 

In [16]:
report = pipeline.report()
report['activations'] = LOADER_CONFIG
report

{'DataPipeline': {'RealAggregateSource': {'allow_incomplete_target': True,
   'allow_multiple_target_activations_in_aggregate': False,
   'filename': '/data/mine/vadeec/merged/ukdale.h5',
   'include_incomplete_target_in_output': True,
   'include_multiple_targets_in_output': False,
   'rng_seed': None,
   'sample_period': 6,
   'seq_length': 256,
   'target_appliance': 'kettle',
   'target_inclusion_prob': 0.5,
   'uniform_prob_of_selecting_each_building': True,
   'windows': {'train': {1: ('2014-01-01', '2014-02-01')},
    'unseen_activations_of_seen_appliances': {1: ('2014-02-02', '2014-02-08')},
    'unseen_appliances': {2: ('2013-06-01', '2013-06-07')}}},
  'SyntheticAggregateSource': {'allow_incomplete_distractors': True,
   'allow_incomplete_target': False,
   'distractor_inclusion_prob': 1.0,
   'include_incomplete_target_in_output': True,
   'rng_seed': None,
   'sample_period': 6,
   'seq_length': 256,
   'target_appliance': 'kettle',
   'target_inclusion_prob': 1.0,
   'unif

In [17]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {
        'dense_layer_0': 100,
        'dense_layer_1':  50,
        'dense_layer_2': 100
    }

    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    
    # Dense layers
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    dense_layer_1 = DenseLayer(
        dense_layer_0, 
        num_units=NUM_UNITS['dense_layer_1']
    )
    dense_layer_2 = DenseLayer(
        dense_layer_1, 
        num_units=NUM_UNITS['dense_layer_2']
    )
    
    # Output
    final_dense_layer = DenseLayer(
        dense_layer_2,
        num_units=target_shape[1] * target_shape[2],
        nonlinearity=None
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [18]:
from neuralnilm.net import Net

batch = pipeline.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer)

In [19]:
from pymongo import MongoClient
client = MongoClient()
client.drop_database('neuralnilm')
db = client.neuralnilm_experiments

In [20]:
from neuralnilm.trainer import Trainer
from neuralnilm.metrics import Metrics

trainer = Trainer(
    net=net,
    data_pipeline=pipeline,
    experiment_id=["0"],
    metrics=Metrics(state_boundaries=[4]),
    learning_rates={0: 1E-2},
    repeat_callbacks=[
        ( 100, Trainer.validate)
    ]
)

INFO:neuralnilm.trainer:[Errno 17] File exists: '~/temp/neural_nilm/output/0'


In [21]:
trainer.fit(100)

INFO:neuralnilm.trainer:Starting training for 100 iterations.
INFO:neuralnilm.trainer:Iteration 0: Change learning rate to 1.0E-02
INFO:neuralnilm.trainer:Compiling train cost function...
INFO:neuralnilm.trainer:Done compiling cost function.


   Update # |  Train cost  | Secs per update | Source ID
------------|--------------|-----------------|-----------
          0 |   0.789326  |    2.883762     |   1

INFO:neuralnilm.net:Compiling deterministic output function...
INFO:neuralnilm.net:Done compiling deterministic output function.



          1 |   0.505262  |    0.002785     |   1
          2 |   0.335411  |    0.004830     |   1
          3 |   0.494606  |    0.008642     |   1
          4 |   0.531029  |    0.007511     |   1
          5 |   1.121243  |    0.189981     |   0
          6 |   0.481029  |    0.128335     |   1
          7 |   0.542973  |    0.177978     |   1
          8 |   1.102234  |    0.133279     |   0
          9 |   0.464178  |    0.177825     |   1
         10 |   1.169114  |    0.145810     |   0
         11 |   1.181369  |    0.098941     |   0
         12 |   0.500052  |    0.248837     |   1
         13 |   0.543184  |    0.160620     |   1
         14 |   0.516095  |    0.213878     |   1
         15 |   0.512686  |    0.187284     |   1
         16 |   1.120034  |    0.077231     |   0
         17 |   1.138080  |    0.128809     |   0
         18 |   1.103928  |    0.134423     |   0
         19 |   0.607981  |    0.096383     |   1
         20 |   1.123067  |    0.130321     |   0

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
INFO:neuralnilm.trainer:Stopped training. Completed 100 iterations.


In [22]:
net.train_iterations

100

In [23]:
from neuralnilm.monitor.monitor import Monitor

mon = Monitor("0")

In [24]:
mon._plot_train_scores()

In [36]:
mon._plot_validation_scores()

In [26]:
mon.validation_metric_names

[u'regression.mean_squared_error',
 u'regression.relative_error_in_total_energy',
 u'regression.mean_absolute_error',
 u'classification_2_state.f1_score',
 u'classification_2_state.recall_score',
 u'classification_2_state.accuracy_score',
 u'classification_2_state.precision_score']

In [27]:
mon.db.validation_scores.distinct(key='source_id')

[0, 1]